In [ ]:
def column_mapping(df_1, df_2, col_1, col_2):
    column_mapping_value = len(df_1[df_1[col_1].isin(df_2[col_2])])
    df_1_mapped_unique_record_indentifier = df_1[df_1[col_1].isin(df_2[col_2])]['unique_record_identifier'].tolist()
    df_2_mapped_unique_record_indentifier = df_2[df_2[col_2].isin(df_1[col_1])]['unique_record_identifier'].tolist()
#     column_mapping_value = df_1[df_1[col_1].isin(df_2[col_2])][col_1].nunique()
    return column_mapping_value, df_1_mapped_unique_record_indentifier, df_2_mapped_unique_record_indentifier

def mapping_matrix(df_1, df_2):
    mapping_arr=[]
    for col_1 in df_1.columns:
        single_column_mapping_arr = []
        single_column_mapping_arr.append(col_1)
        for col_2 in df_2.columns:
            column_mapping_value = column_mapping(df_1, df_2, col_1, col_2)
            single_column_mapping_arr.append(column_mapping_value)
        mapping_arr.append(single_column_mapping_arr)
        
    # Create a dataset
    columns = df_2.columns.tolist()
    columns.insert(0, 'df_1')
    matrix_df = pd.DataFrame(mapping_arr, columns=columns)
    matrix_df.set_index("df_1", inplace=True)
    matrix_df.to_csv('mapping_matrix.csv')

In [ ]:
def estimated_time(one_field_runtime, no_of_columns):
    current_time = datetime.datetime.now()
    hr = current_time.hour
    minute = current_time.minute
    sec = current_time.second
    print("one_field_runtime", one_field_runtime)
    print("no_of_columns", no_of_columns)
    total_runtime = (one_field_runtime * no_of_columns)/60
    print("total_runtime", total_runtime)
    minute = minute + total_runtime
    if minute >= 60:
        hr+=1
        minute = minute - 60
        if minute == 60:
            minute=='00'
    if len(str(minute))==1:
        minute = '0'+str(minute)
    print("\t\t\t\tEstimated Time: " + str(hr) + ":" + str(minute))
    

def random_mapping(df_1, df_2, coverage=0, final_arr=[]):
    no_of_columns = len(df_1.columns.tolist())
    cnt=0
    df_1_mapped_unique_indentifier_arr = []
    df_2_mapped_unique_indentifier_arr = []
    for x in df_1.columns:
        if x != 'unique_record_identifier':
            if cnt == 0:
                t3 = time.time()
            if cnt == 1:
                t4 = time.time()
                one_field_runtime = round((t4-t3))
    #             estimated_time(one_field_runtime, no_of_columns)
            cnt+=1
            if(cnt%10==0):
                print("Checkpoint: " + str(cnt) + "/" + str(no_of_columns))
            mapping_value = 0
            mapping_df_1_column = mapping_df_2_column= ''
            for col_1 in df_1.columns:
                if col_1 != 'unique_record_identifier':
                    for col_2 in df_2.columns:
                        if col_2 != 'unique_record_identifier':
                            column_mapping_value, df_1_mapped_indentifier, df_2_mapped_indentifier = column_mapping(df_1, df_2, col_1, col_2)
                            if column_mapping_value > mapping_value:
                                mapping_value = column_mapping_value
                                df_1_mapped_unique_indentifier = df_1_mapped_indentifier
                                df_2_mapped_unique_indentifier = df_2_mapped_indentifier
                                mapping_df_1_column = col_1
                                mapping_df_2_column = col_2
            if (mapping_df_1_column != ''):
                final_mapping = mapping_df_1_column + " ---> " + mapping_df_2_column + " = " + str(mapping_value)
                df_1_mapped_unique_indentifier_arr.append(df_1_mapped_unique_indentifier)
                df_2_mapped_unique_indentifier_arr.append(df_2_mapped_unique_indentifier)
#                 print(final_mapping)
                final_arr.append(final_mapping)
                temp_1 = pd.DataFrame(df_1[mapping_df_1_column].unique().tolist(), columns={mapping_df_1_column}, dtype='O')
                temp_2 = pd.DataFrame(df_2[mapping_df_2_column].unique().tolist(), columns={mapping_df_2_column}, dtype='O')
                temp_df=pd.merge(temp_1, temp_2, left_on=mapping_df_1_column, right_on=mapping_df_2_column)
                df_1_length_before = len(df_1)
                df_1 = df_1[~df_1[mapping_df_1_column].isin(temp_df[mapping_df_1_column])]
                df_2 = df_2[~df_2[mapping_df_2_column].isin(temp_df[mapping_df_1_column])]
                df_1_length_after = len(df_1)
                coverage += df_1_length_before - df_1_length_after
    return coverage, df_1, df_2, final_arr, df_1_mapped_unique_indentifier_arr, df_2_mapped_unique_indentifier_arr

In [ ]:
def field_remove(df, remove_field_arr):
    df.drop(remove_field_arr, errors='ignore', inplace=True, axis=1)
    return df

In [ ]:
def clean_up(df, i=0):
    df.columns = df.columns.str.lower()
    df = df.dropna(how='all', axis=1)
    df = df.dropna(how='all')
    for col in df.columns:
        df[col] = df[col].map(str)
        df[col] = df[col].apply(lambda x: x.lower())
#         df[col] = df[col].replace('nan', str(i)+"_nan")
#         i+=1
    audit_field = ['source_type', 'source_name', 'original_string', 'source_uuid', 'event_timestamp_epoch', 'event_timeoffset', 'event_timestamp_iso_client_timezone', 'event_timestamp_day', 'event_timestamp_month', 'event_timestamp_year', 'event_timestamp_week_day', 'event_timestamp_week_index', 'event_timestamp_hour', 'event_timestamp_minute', 'event_timestamp_date', 'parsed_timestamp', 'ingested_timestamp', 'analysis_period_start', 'analysis_period_end']
    for col in df.columns:
        unique_col_values = str(df[col].unique().tolist())
        if ((unique_col_values == "['false', 'true']") | (unique_col_values == "['true', 'false']") | (unique_col_values == "['nan', 'true', 'false']") |(unique_col_values == "['nan', 'false', 'true']") | (unique_col_values== "['false']") | (unique_col_values== "['nan', 'false']") | (unique_col_values== "['true']") | (unique_col_values== "['nan', 'true']") | (unique_col_values== "['true', 'nan']") | (unique_col_values== "['false', 'nan']") | (unique_col_values == "['true', 'false', 'nan']") | (unique_col_values == "['false', 'true', 'nan']")):
            audit_field.append(col)
        if ((('unnamed' in col) | ('epoch' in col) | ('time' in col) | ('date' in col)) & (col not in audit_field)):
            audit_field.append(col)  
    df = field_remove(df, audit_field)
    for col in df.columns:
        df[col] = df[col].replace('nan', str(i)+"_nan")
        i+=1
#     df.drop(audit_field, errors='ignore', inplace=True, axis=1)
    return df, i

In [ ]:
def columns_to_keep(df, col_to_keep):
    col_to_keep = col_to_keep[1:-1].split(",")
    col_to_keep = [x.strip() for x in col_to_keep]
    df = df[col_to_keep]
    return df

In [ ]:
def mapping_summary(df_1_name, df_1, coverage, total_duplicate_records, time_taken=0):
#     print("\n***************************************************************************\n")
    print(colored("\n\nMAPPING SUMMARY:", attrs=['bold']))
    print("Number of records in ("+ df_1_name + "): " + str(len(df_1)))
    print("Number of records matched: ", coverage)
    print("Number of records not matched: ", len(df_1) - coverage)
    print("Number of duplicate records: ", total_duplicate_records)
    print("Coverage:", str("{:.2f}".format(round((coverage/len(df_1))*100)) + "%"))
    print("Accuracy:", str("{:.2f}".format(((len(df_1) - total_duplicate_records)/len(df_1))*100) + "%"))
    if time_taken!=0:
        print("Time taken to run:", str(time_taken) + "mins")

In [ ]:
def mapping(final_arr, df_1_name, df_2_name):
    print(colored("\nMAPPING DETAILS ("+ df_1_name + " "+"--->"+ " "+ df_2_name + "):", attrs=['bold']))
    for x in final_arr:
        print(x)

In [ ]:
def duplicate_field_details(mapping_type, final_arr):
    mapping_df_1_col_list = []
    mapping_df_2_col_list = []
    mapping_df_1_duplicate_col_list = []
    mapping_df_2_duplicate_col_list = []
    
    for mapping_col in final_arr:
        mapping_df_1_col_list.append(mapping_type +'_secondary_mapped_' +  mapping_col.split(" ")[0])
        mapping_df_2_col_list.append(mapping_type +'_primary_mapped_' +  mapping_col.split(" ")[2])
        mapping_df_1_duplicate_col_list.append(mapping_type +'_secondary_mapped_' +  mapping_col.split(" ")[0] + '_duplicate')
        mapping_df_2_duplicate_col_list.append(mapping_type +'_primary_mapped_' +  mapping_col.split(" ")[2] + '_duplicate')
        
    mapped_arr = [mapping_type + '_secondary_mapped', mapping_type + '_primary_mapped']
    duplicate_arr = [mapping_type + '_secondary_duplicate', mapping_type + '_primary_duplicate']
    total_duplicate_records = []
    
    globals()[mapped_arr[0]] = pd.DataFrame()
    globals()[duplicate_arr[0]] = []
    globals()[mapped_arr[1]] = pd.DataFrame()
    globals()[duplicate_arr[1]] = []

    for i in range(len(final_arr)):
        df_1_id = df_1_mapped_unique_indentifier[i]
        df_2_id = df_2_mapped_unique_indentifier[i]
        mapped_df_1 = df_1_original[df_1_original['unique_record_identifier'].isin(df_1_id)]
        mapped_df_2 = df_2_original[df_2_original['unique_record_identifier'].isin(df_2_id)]
        globals()[mapped_arr[0]] = pd.concat([globals()[mapped_arr[0]], mapped_df_1])
        globals()[mapped_arr[1]] = pd.concat([globals()[mapped_arr[1]], mapped_df_2])
        globals()[mapping_df_1_col_list[i]] = mapped_df_1
        globals()[mapping_df_2_col_list[i]] = mapped_df_2
        
        arr=[]
        column_name_secondary = final_arr[i].split(" ")[0]
        column_name_primary = final_arr[i].split(" ")[2]      
        globals()[mapping_df_1_duplicate_col_list[i]] = globals()[mapping_df_1_col_list[i]].groupby(column_name_secondary).filter(lambda x: len(x) > 1)[column_name_secondary].unique().tolist()
        if len(globals()[mapping_df_1_duplicate_col_list[i]]) > 0:
            globals()[duplicate_arr[0]].append(column_name_secondary)
            total_duplicate_records.extend(globals()[mapping_df_1_duplicate_col_list[i]])
            
        globals()[mapping_df_2_duplicate_col_list[i]] = globals()[mapping_df_2_col_list[i]].groupby(column_name_primary).filter(lambda x: len(x) > 1)[column_name_primary].unique().tolist()
        if len(globals()[mapping_df_2_duplicate_col_list[i]]) > 0:
            globals()[duplicate_arr[1]].append(column_name_primary)
            total_duplicate_records.extend(globals()[mapping_df_2_duplicate_col_list[i]])
    return total_duplicate_records

def duplicate_records_summary(mapping_type):
    print(colored("\n\nDUPLICATE SUMMARY:", attrs=['bold']))
    if mapping_type == 'random':
        print(mapping_type[0].upper() + mapping_type[1:] + " " + "Primary Source Duplicate Fields:", random_primary_duplicate)
        print(mapping_type[0].upper() + mapping_type[1:] + " " + "Secodnary Source Duplicate Fields:", random_secondary_duplicate)
    else:
        print(mapping_type[0].upper() + mapping_type[1:] + " " + "Primary Source Duplicate Fields:", priority_primary_duplicate)
        print(mapping_type[0].upper() + mapping_type[1:] + " " + "Secodnary Source Duplicate Fields:", priority_secondary_duplicate)
        
    if mapping_type == 'random':
        color = 'green'
    else:
        color = 'red'
    print(colored("\n" + "*"*126 + "\n", color, attrs=['bold']))
    

In [ ]:
def source_of_truth_explode(df_2):
    columns_to_be_added = []
    for col in df_2.columns:
        if (("source_of_truth" in col) & (len(col) > 14)):
            df_2[col].apply(lambda x: columns_to_be_added.extend(list(list(zip(*x))[0])))
    return set(columns_to_be_added)

In [ ]:
import matplotlib.pyplot as plt
from termcolor import colored
import seaborn as sns
import pandas as pd
import numpy as np
import datetime
import warnings
import json
import time
import ast

warnings.filterwarnings("ignore")

t0 = time.time()

f = open (r'C:\Users\nevintoms\Desktop\Field_Mapping_automation\Input_Config\input_details.json', "r")
  
# Reading from file
data = json.loads(f.read())

secondary_path = data['secondary_path']
primary_path = data['primary_path']
# secondary_path = secondary_path.replace("\\", "\\")
# primary_path = primary_path.replace("\\", "\\")

secondary_path_file_type = secondary_path.split(".")[-1]
primary_path_file_type = primary_path.split(".")[-1]
if secondary_path_file_type == "csv":
    df_1 = pd.read_csv(secondary_path)
elif secondary_path_file_type == "json":
    df_1 = pd.read_json(secondary_path)
else:
    df_1 = pd.read_parquet(secondary_path)
    
if primary_path_file_type == "csv":
    df_2 = pd.read_csv(primary_path)
elif primary_path_file_type == "json":
    df_2 = pd.read_json(primary_path)
else:
    df_2 = pd.read_parquet(primary_path)
    
df_1 = df_1.replace(r'^\s*$', np.nan, regex=True)
df_2 = df_2.replace(r'^\s*$', np.nan, regex=True)
df_1 = df_1.fillna(value=np.nan)
df_2 = df_2.fillna(value=np.nan)

matrix = data['matrix']
random = data['random']
priority = data['priority_from_secondary']
priority_random = data['priority_from_secondary_random']
col_to_be_deleted_df_1 = data['col_to_be_ignored_from_secondary']
col_to_be_deleted_df_2 = data['col_to_be_ignored_from_primary']
col_to_keep_df_1 = data['col_to_considered_from_secondary']
col_to_keep_df_2 = data['col_to_considered_from_primary']
secondary_duplicate_update = data['secondary_duplicate_update']
primary_duplicate_update = data['primary_duplicate_update']
inventory_remove_source_name = data["inventory_remove_source_name"]
inventory_source_of_truth_explode = data["inventory_source_of_truth_explode"]

if inventory_source_of_truth_explode=="True":
    print("starting")
    columns_to_be_added = source_of_truth_explode(df_2)

# df_1_name = secondary_path.split("\\")[-1]
# df_2_name = primary_path.split("\\")[-1]

# df_1.loc[:, 'unique_record_identifier'] = [str(i) for i in range(len(df_1))]
# df_2.loc[:, 'unique_record_identifier'] = [str(i) for i in range(len(df_2))]

# df_1_original = df_1.copy()
# df_2_original = df_2.copy()

# if len(inventory_remove_source_name) > 2:
#     df_2 = df[df_2['source_of_truth'] != inventory_remove_source_name]

# def arr_length_greater_than_zero(arr):
#     if len(arr[1:-1].split(",")) > 0:
#         return True
#     else:
#         return False
    
# if (arr_length_greater_than_zero(col_to_keep_df_1)):
#     col_to_keep_df_1 = col_to_keep_df_1[:-1]+',unique_record_identifier'+']'
#     df_1 = columns_to_keep(df_1, col_to_keep_df_1)
    
# if (arr_length_greater_than_zero(col_to_keep_df_2)):
#     col_to_keep_df_2 = col_to_keep_df_2[:-1]+',unique_record_identifier'+']'
#     df_2 = columns_to_keep(df_2, col_to_keep_df_2)
    
# if (arr_length_greater_than_zero(col_to_be_deleted_df_1)):
#     col_to_be_deleted = col_to_be_deleted_df_1[1:-1].split(",")
#     col_to_be_deleted = [x.strip() for x in col_to_be_deleted]
#     df_1 = field_remove(df_1, col_to_be_deleted)

# if (arr_length_greater_than_zero(col_to_be_deleted_df_2)):
#     col_to_be_deleted = col_to_be_deleted_df_2[1:-1].split(",")
#     col_to_be_deleted = [x.strip() for x in col_to_be_deleted]
#     df_2 = field_remove(df_2, col_to_be_deleted)
    
# if len(ast.literal_eval(secondary_duplicate_update)) > 0:
#     secondary_duplicate_update = ast.literal_eval(secondary_duplicate_update)
#     for duplicate_update in list(secondary_duplicate_update.keys()):
#         df_1[secondary_duplicate_update[duplicate_update]].fillna('1840-09-23T12:28:49.664010Z')
#         df_1 = df_1.sort_values(secondary_duplicate_update[duplicate_update]).groupby(duplicate_update).tail(1)
          
# if len(ast.literal_eval(primary_duplicate_update)) > 0:
#     primary_duplicate_update = ast.literal_eval(primary_duplicate_update)
#     for duplicate_update in list(primary_duplicate_update.keys()):
#         df_2[primary_duplicate_update[duplicate_update]].fillna('1840-09-23T12:28:49.664010Z')
#         df_2 = df_2.sort_values(primary_duplicate_update[duplicate_update]).groupby(duplicate_update).tail(1)

# # df_2 = df_2.sort_values('last_logon').groupby('employee_id').tail(1)

# print("Message: Cleaning Started")
# df_1, i = clean_up(df_1)
# df_2, i = clean_up(df_2, i)
# print("Message: Cleaning Completed\n")

# if matrix == 'True':
#     mapping_matrix(df_1,df_2)
#     print("Message: Mapping Matrix Completed\n")
    
# if random == 'True':
#     print(colored("\n" + "*"*56 + "RANDOM MAPPING" + "*"*56, 'green', attrs=['bold']))
#     final_arr=[]
#     coverage, df_1_not_mapped, df_2_not_mapped, final_arr, df_1_mapped_unique_indentifier, df_2_mapped_unique_indentifier = random_mapping(df_1, df_2, 0, final_arr)
#     random_secondary_not_mapped = df_1_original[df_1_original['unique_record_identifier'].isin(df_1_not_mapped['unique_record_identifier'])]
#     random_primary_not_mapped = df_2_original[df_2_original['unique_record_identifier'].isin(df_2_not_mapped['unique_record_identifier'])]
#     t1 = time.time()
#     time_taken = round((t1-t0)/60,2)
#     mapping(final_arr, df_1_name, df_2_name)
#     total_duplicate_records = duplicate_field_details('random', final_arr)
#     mapping_summary(df_1_name, df_1, coverage, len(total_duplicate_records))
#     duplicate_records_summary('random')
    
# priority_temp = priority[1:-1].split()
# if ((len(priority_temp ) > 0)):
#     print(colored("\n" + "*"*55 + "PRIORITY MAPPING" + "*"*55, 'red', attrs=['bold']))
#     priority = priority[1:-1].split(",")
#     priority = list(map(lambda x: x.lower(), priority))
#     df_1_not_mapped = df_1.copy()
#     df_2_not_mapped = df_2.copy()
#     coverage=0
#     final_arr=[]
#     df_1_mapped_unique_indentifier = []
#     df_2_mapped_unique_indentifier = []
#     for priority_item in priority:
#         priority_item = priority_item.strip()
#         priority_item = priority_item+","+'unique_record_identifier'
#         priority_item_modified = "[" + priority_item + "]"
#         df_1_priority = columns_to_keep(df_1_not_mapped, priority_item_modified)
#         coverage, df_1_not_mapped, df_2_not_mapped, final_arr, df_1_mapped_unique_indentifier_priority, df_2_mapped_unique_indentifier_priority = random_mapping(df_1_priority, df_2_not_mapped, coverage, final_arr)
#         df_1_not_mapped = df_1[df_1['unique_record_identifier'].isin(df_1_not_mapped['unique_record_identifier'])]
      
#         if len(df_1_mapped_unique_indentifier_priority) > 0:
#             df_1_mapped_unique_indentifier.extend(df_1_mapped_unique_indentifier_priority)
#         if len(df_2_mapped_unique_indentifier_priority) > 0:
#             df_2_mapped_unique_indentifier.extend(df_2_mapped_unique_indentifier_priority)
#     if priority_random == 'True':
#         coverage, df_1_not_mapped, df_2_not_mapped, final_arr, df_1_mapped_unique_indentifier_priority, df_2_mapped_unique_indentifier_priority = random_mapping(df_1_not_mapped, df_2_not_mapped, coverage, final_arr)
#         df_1_mapped_unique_indentifier.extend(df_1_mapped_unique_indentifier_priority)
#         df_2_mapped_unique_indentifier.extend(df_2_mapped_unique_indentifier_priority)
    
#     priority_secondary_not_mapped = df_1_original[df_1_original['unique_record_identifier'].isin(df_1_not_mapped['unique_record_identifier'])]
#     priority_primary_not_mapped = df_2_original[df_2_original['unique_record_identifier'].isin(df_2_not_mapped['unique_record_identifier'])]
#     mapping(final_arr, df_1_name, df_2_name)
#     total_duplicate_records = duplicate_field_details('priority', final_arr)
#     mapping_summary(df_1_name, df_1, coverage, len(total_duplicate_records))
#     duplicate_records_summary('priority')
    
# print(colored("\nVariable Name Details:", attrs=['bold']))
# print("Mapped Details of Primary/Secondary Source: (random/priority)_(primary/secondary)_mapped")
# print("Unmapped Deatails of Primary/Secodnary Source: (random/priority)_(primary/secondary)_not_mapped")
# print("Individual Mapping Details: (random/priority)_(primary/secondary)_mapped_(mapping_field_name)")
# print("Duplicate mapping details: (random/priority)_(primary/secondary)_mapped_(mapping_field_name)_duplicate")

In [ ]:
print(columns_to_be_added)

In [ ]:
a=[('SF', 'nevin'), ('AD', 'TOMS')]

In [ ]:
zip(*[('a', 1), ('b', 2), ('c', 3), ('d', 4)])

In [ ]:
df_2 = pd.read_parquet(r"C:\Users\Public\Downloads\SummaryDM\12Oct2022\INV")

In [ ]:
columns_to_be_added = []
for col in df_2.columns:
    if (("source_of_truth" in col) & (len(col) > 14)):
        print(col)
        for x in df_2[col]:
            if x is not None:
                print(x)
                columns_to_be_added.extend(list(list(zip(*x))[0]))
#         df_2[col].apply(lambda x: columns_to_be_added.extend(list(list(zip(*x))[0])))

In [ ]:
set(columns_to_be_added)

In [ ]:
df_2['object_name_with_source_of_truth']

In [ ]:
for x in zip(*a):
    print(x[1])

In [ ]:
tu = set(list(zip(*a))[0])

In [ ]:
tu.add("AD")

In [ ]:
tu

In [ ]:
tuple(tu)

In [ ]:
priority_primary_mapped_employee_id_duplicate

In [ ]:
priority_secondary_mapped[priority_secondary_mapped['employee_id']=='14200']

In [ ]:
priority_primary_mapped[priority_primary_mapped['employee_id']=='14200'][['object_name', 'last_logon']]